SQL и PANDAS

### 1.1 

Подготовка данных:

In [3]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

df = pd.read_sql_query("select * from public.ratings limit 5;", conn)
df.head()

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('course_purchases',)]


,userid,movieid,rating,timestamp
0,4463,924,4.0,1100831685
1,4463,1025,3.5,1100755105
2,4463,1073,4.0,1100825419
3,4463,1101,3.0,1120809183
4,4463,1127,3.5,1100753902


Аналитика Pandas и работа c DB PostgreSQL

In [5]:
import psycopg2

pg_connection = {
    "host": "pass",
    "port": 5432,
    "dbname": "db_ds",
    "user": "readonly",
    "password": "pass"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

table_names = [i[0] for i in tables_data]
for table in table_names:
        sql_str = "SELECT COUNT(*) FROM public.{} LIMIT 1;".format(table)
        cursor.execute(sql_str)
        row_count = [a for a in cursor.fetchall()][0][0]
        print("table {}:{}".format(table, row_count))

cursor.close()

table keywords:92838
table links:91686
table ratings:1555552
table course_purchases:154


In [1]:
import psycopg2

pg_connection = {
    "host": "pass",
    "port": 5432,
    "dbname": "db_ds_s",
    "user": "readonly",
    "password": "pass"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

cursor.execute("SELECT * FROM keywords LIMIT 5;")
print(cursor.fetchall())
cursor.close()

[(862, "[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"), (8844, '[{\'id\': 10090, \'name\': \'board game\'}, {\'id\': 10941, \'name\': \'disappearance\'}, {\'id\': 15101, \'name\': "based on children\'s book"}, {\'id\': 33467, \'name\': \'new home\'}, {\'id\': 158086, \'name\': \'recluse\'}, {\'id\': 158091, \'name\': \'giant insect\'}]'), (15602, "[{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': 'best friend'}, {'id': 179431, 'name': 'duringcreditsstinger'}, {'id': 208510, 'name': 'old men'}]"), (31357, "[{'id': 818, 'name': 'based on novel'}, {'id': 10131, 'name': 'interracial relationship'}, {'id': 14768, 'name': 'single mother'}, {'id': 15160, 'name': 'divorce'}, {'id': 33455, 'name': 'chick flick'}]"), (11862,

In [4]:
import psycopg2

pg_connection = {
    "host": "pass",
    "port": 5432,
    "dbname": "db_ds",
    "user": "readonly",
    "password": "pass"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM ratings LIMIT 10;"
cursor.execute(sql_str)
conn.commit()
table_rating_fields = []
table_rating_fields = [desc[0] for desc in cursor.description]
print("Поля таблицы ratings: %s" % table_rating_fields)

cursor.close()

Поля таблицы ratings: ['userid', 'movieid', 'rating', 'timestamp']


Создание запроса в SQL 

In [8]:
import psycopg2

pg_connection = {
    "host": "pass",
    "port": 5432,
    "dbname": "db_ds\",
    "user": "readonly",
    "password": "pass"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()
cursor.execute("SELECT * FROM ratings;")
conn.commit()
table_rating_fields = []
table_rating_fields = [desc[0] for desc in cursor.description]
cursor.execute("SELECT * FROM ratings ORDER BY movieid LIMIT 10 ;")
conn.commit()
tableRatings = []
tableRatings = cursor.fetchall()
boardRatings = pd.DataFrame(tableRatings)
boardRatings.columns = table_rating_fields
print(boardRatings)
cursor.execute("SELECT * FROM links;")
conn.commit()
table_links_fields2 = []
table_links_fields2 = [desc[0] for desc in cursor.description]
cursor.execute("SELECT * FROM links ORDER BY movieid LIMIT 10;")
conn.commit()
tableLinks = []
tableLinks = cursor.fetchall()
boardLinks = pd.DataFrame(tableLinks)
boardLinks.columns = table_links_fields2
print(boardLinks)
print("Джойнить по полю movieid!")
print()
sqlRequest = '''SELECT COUNT(LL.movieid) 
                FROM  (SELECT movieid 
                    FROM (SELECT links.movieid, COUNT(tmdbid) AS tmd 
                        FROM links GROUP BY links.movieid) AS L) AS LL
                    
                LEFT OUTER JOIN (SELECT movieid, CR 
                    FROM (SELECT ratings.movieid, COUNT(rating) AS CR 
                        FROM ratings GROUP BY ratings.movieid) AS R)AS RR  ON LL.movieid = RR.movieid 
                WHERE CR IS NULL;'''
cursor.execute(sqlRequest) 
conn.commit()
print("Фильмов не имеют рейтинга: ", cursor.fetchall())
cursor.close()

   userid  movieid  rating   timestamp
0    4490        1     5.0   994047533
1    4493        1     4.0  1322208878
2    4481        1     4.5  1079905235
3    4489        1     4.0   874983794
4    4492        1     5.0  1258409444
5    4473        1     4.0   868396522
6    4467        1     5.0   860090531
7    4477        1     3.0   860583991
8    4476        1     5.0   833644404
9    4495        1     5.0   833880174
   movieid   imdbid tmdbid
0        1  0114709    862
1        1  0114709    862
2        2  0113497   8844
3        2  0113497   8844
4        3  0113228  15602
5        3  0113228  15602
6        4  0114885  31357
7        4  0114885  31357
8        5  0113041  11862
9        5  0113041  11862
Джойнить по полю movieid!

Фильмов не имеют рейтинга:  [(28333,)]


Работа с БД: MongoDB

In [9]:
from pymongo import MongoClient

mongo_connection = {
    "host": "pass",
    "port": 27017,
    "user": "pass",
    "password": "pass",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Коллекции, доступные в MongoDB: ['tags', 'users']
Число документов в коллекции 158680


In [24]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}
mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'})
cursor_items = [item for item in mongo_cursor]
print("Количество контента с тэгом 'toy': %s\n" % len(cursor_items))

Количество контента с тэгом 'toy': 19



In [74]:
from pymongo import MongoClient
mongo_connection = {
    "host": "pass",
    "port": 27017,
    "user": "pass",
    "password": "pass",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]
pipline = [
    {"$group":
        {"_id": "$name",
         "tag_count":
            {"$sum": 1}
         }
     },
    {"$sort":
        {"tag_count": 1}
     },
    {"$limit": 10}
]

table1 = []
table = ([i for i in collection.aggregate(pipline)])
for i in range(len(table)):
    table1.append(table[i]['_id'])

print(table1)

['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']


In [171]:
import psycopg2
import random
import json

pg_connection = {
    "host": "pass",
    "port": 5432,
    "dbname": "db_ds",
    "user": "readonly",
    "password": "pass"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

cursor.execute("""SELECT movieid FROM ratings ORDER BY rating DESC LIMIT 10;""")
conn.commit()
popularMovieID = cursor.fetchall()

slovar = {}
spisok = []
for i in range(len(popularMovieID)):
    slovar['movieID'] = popularMovieID[i][0]
    slovar['popularity'] = popularMovieID[i][0] * random.randint(2,3)
    spisok.append(dict(slovar))
spisok = json.dumps(spisok)
spisok = spisok.replace('[', '')
spisok = spisok.replace(']', '')

with open('content_popularity.json ', 'w') as outfile:
    json.dump(spisok, outfile)